In [77]:
import pandas as pd

In [78]:
df_crypto = pd.read_csv('binance_crypto_data_20250327.csv')
df_OHLC = pd.read_csv('binance_ohlc_data_20250327.csv')


In [79]:
combined_df = pd.merge(df_OHLC, df_crypto, left_on=['timestamp', 'symbol'], right_on=['date', 'asset'])

In [80]:
combined_df = combined_df.drop('date', axis=1)

In [81]:
df_gold = pd.read_csv('Gold_price_averages_monthly.csv')

In [82]:


df_gold['date'] = pd.to_datetime(df_gold['date'])
df_gold = df_gold[df_gold['date'] >= pd.to_datetime('2021-12-31')]

/var/folders/vr/qwxtvrr17fn2qzlm0z4rq4br0000gn/T/ipykernel_98313/2657357993.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_gold['date'] = pd.to_datetime(df_gold['date'])


In [83]:
df_gold = df_gold[['date', 'USD']]
df_gold.head()

,date,USD
527,2021-12-31,"1,786.7"
528,2022-01-31,"1,816.8"
529,2022-02-28,"1,856.3"
530,2022-03-31,"1,947.8"
531,2022-04-29,"1,933.9"


In [84]:
df_gold["date"] = pd.to_datetime(df_gold["date"])
df_gold.set_index("date", inplace=True)

# Create a daily date range covering the full period up to March 24, 2025
date_range = pd.date_range(start=df_gold.index.min(), end="2025-03-24", freq='D')

df_gold_daily = df_gold.reindex(date_range, method='ffill')
df_gold_daily.reset_index(inplace=True)
df_gold_daily.rename(columns={"index": "date"}, inplace=True)

print(df_gold_daily)

           date      USD
0    2021-12-31  1,786.7
1    2022-01-01  1,786.7
2    2022-01-02  1,786.7
3    2022-01-03  1,786.7
4    2022-01-04  1,786.7
...         ...      ...
1175 2025-03-20  2,894.7
1176 2025-03-21  2,894.7
1177 2025-03-22  2,894.7
1178 2025-03-23  2,894.7
1179 2025-03-24  2,894.7

[1180 rows x 2 columns]


In [85]:
df_gold_daily.rename(columns={'USD': 'USD_Price_Gold_Monthly'}, inplace=True)
print(df_gold_daily)

           date USD_Price_Gold_Monthly
0    2021-12-31                1,786.7
1    2022-01-01                1,786.7
2    2022-01-02                1,786.7
3    2022-01-03                1,786.7
4    2022-01-04                1,786.7
...         ...                    ...
1175 2025-03-20                2,894.7
1176 2025-03-21                2,894.7
1177 2025-03-22                2,894.7
1178 2025-03-23                2,894.7
1179 2025-03-24                2,894.7

[1180 rows x 2 columns]


In [86]:
combined_df['timestamp'] = pd.to_datetime(combined_df['timestamp'])
combined_df = combined_df.merge(df_gold_daily, left_on='timestamp', right_on='date', how='left')

In [87]:
combined_df['USD_Price_Gold_Monthly'] = combined_df['USD_Price_Gold_Monthly'].str.replace(',', '').astype(float)
print(combined_df)

           symbol  timestamp      open      high       low     close  \
0        BTC/USDT 2022-03-24  42882.76  44220.89  42560.46  43991.46   
1        BTC/USDT 2022-03-25  43991.46  45094.14  43579.00  44313.16   
2        BTC/USDT 2022-03-26  44313.16  44792.99  44071.97  44511.27   
3        BTC/USDT 2022-03-27  44511.27  46999.00  44421.46  46827.76   
4        BTC/USDT 2022-03-28  46827.76  48189.84  46663.56  47122.21   
...           ...        ...       ...       ...       ...       ...   
108885  PAXG/USDT 2025-03-20   3065.00   3072.00   3047.00   3057.00   
108886  PAXG/USDT 2025-03-21   3059.00   3100.00   3031.00   3039.00   
108887  PAXG/USDT 2025-03-22   3039.00   3051.00   3039.00   3050.00   
108888  PAXG/USDT 2025-03-23   3051.00   3062.00   3042.00   3054.00   
108889  PAXG/USDT 2025-03-24   3054.00   3058.00   3002.00   3019.00   

           volume_x     price     volume_y    market_cap  daily_return  \
0       56195.12374  43991.46  56195.12374  2.472106e+09     

In [89]:
combined_df = combined_df.drop('date', axis=1)

In [90]:
combined_df.to_csv('Combined_crypto_OHLC_gold.csv', index=False)